# 🚀 **JLSatiro Clipper AI - V24.0 Stable Edition**

**Modo Simplificado & Rápido**. Instalação limpa a cada execução para garantir zero erros.

### ⚡ **Recursos:**
- **Smart Crop & Transcrição GPU**
- **Salva Resultados no Drive**
- **Instalação Otimizada (Wheels)**

---

In [ ]:
#@title 1. 🛠️ Instalação Rápida (Standard Mode)
#@markdown Instalação limpa e direta. Sem cache complexo para evitar erros.

import os
import subprocess
import sys
import shutil

print("💎 [JLSatiro] Iniciando Setup V24.0 (Modo Estável)...")

# 1. Mount Drive (Apenas para salvar os vídeos finais/logins)
print("☁️ [1/5] Conectando Drive (Para salvar vídeos)...")
try:
    from google.colab import drive
    drive.mount('/content/drive')
    CONFIG_DIR = "/content/drive/MyDrive/JLSatiro_Config"
    os.makedirs(CONFIG_DIR, exist_ok=True)
except:
    print("⚠️ Drive não conectado. Vídeos ficarão apenas no temporário.")
    CONFIG_DIR = None

# 2. System Deps
print("📦 [2/5] Instalando FFmpeg & ImageMagick...")
subprocess.run("apt-get update -qq && apt-get install -y ffmpeg imagemagick libsndfile1 sox -qq > /dev/null", shell=True)
if os.path.exists("/etc/ImageMagick-6/policy.xml"):
    subprocess.run("sed -i 's/none/read,write/g' /etc/ImageMagick-6/policy.xml", shell=True)

# 3. Clone/Update Repo
print("📂 [3/5] Baixando Código JLSatiro...")
repo_url = "https://github.com/noviso123/JLSatiroClipperAI.git"
repo_dir = "/content/JLSatiroClipperAI"

if os.path.exists(repo_dir): shutil.rmtree(repo_dir) # Force clean pull
subprocess.run(f"git clone {repo_url} {repo_dir}", shell=True)

# 4. Python Deps (Fast Wheels)
print("🐍 [4/5] Instalando Bibliotecas (Modo Turbo - Wheels)...")

# Llama GPU Wheel (Critical Speedup)
subprocess.run('pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121 --no-cache-dir', shell=True)

# Install requirements.txt (Includes all pins: gradio, hf_hub<0.25, etc)
req_path = os.path.join(repo_dir, "requirements.txt")
subprocess.run(f"{sys.executable} -m pip install -r {req_path} --no-cache-dir", shell=True)

# 5. Restore Configs (Login Only - Safe)
if CONFIG_DIR:
    print("♻️ [5/5] Restaurando Logins...")
    for f in ["client_secret.json", "token.pickle", "cookies.txt"]:
        src = os.path.join(CONFIG_DIR, f)
        dst = os.path.join(repo_dir, f)
        if os.path.exists(src): shutil.copy(src, dst)

print("\n✨ INSTALAÇÃO LIMPA E COMPLETA! Pode iniciar.")

In [ ]:
#@title 2. 🚀 Iniciar JLSatiro Clipper AI

import os
import shutil

os.chdir("/content/JLSatiroClipperAI")

# MediaPipe Fix (Always apply on clean install)
video_engine_path = "backend/video_engine.py"
if os.path.exists(video_engine_path):
    with open(video_engine_path, 'r', encoding='utf-8') as f: c = f.read()
    if "from mediapipe import solutions" in c:
        c = c.replace("from mediapipe import solutions\n\n        mp_face_detection = solutions.face_detection", "import mediapipe as mp\n\n        mp_face_detection = mp.solutions.face_detection")
        c = c.replace("from mediapipe import solutions\n\n        mp_face = solutions.face_detection", "import mediapipe as mp\n\n        mp_face = mp.solutions.face_detection")
        with open(video_engine_path, 'w', encoding='utf-8') as f: f.write(c)

try:
    !python app.py
except KeyboardInterrupt: pass
finally:
    # Simple Backup of logins only
    CONFIG_DIR = "/content/drive/MyDrive/JLSatiro_Config"
    if os.path.exists(CONFIG_DIR):
        print("\n💾 Salvando logins...")
        for f in ["client_secret.json", "token.pickle", "cookies.txt"]:
            if os.path.exists(f): shutil.copy(f, os.path.join(CONFIG_DIR, f))